## Deal with quantitative queries on Dandi Search

In [1]:
# Basic: Deal with finding dandisets with 2 or more species. Filter via Qdrnat api
from qdrant_client import QdrantClient as Qdrant
from qdrant_client.models import models
import os

# ---------------------

QUERY = "Are there any dandisets that contain no more than 2 species?"

#Ex: Which dandisets have more than X subjects/neurons from brain region Y?
#Ex: Show me dandisets with the most species.

# ---------------------


In [8]:
# Approach #1: LLM
from langchain.llms.openai import OpenAIChat
import json

template = """
Your task is to extract a numerical value that represents a specific asset along with the asset type itself from a given user query.
An asset type can only be one of: species, approach, variables_measured, measurement_technique.
Choose a comparison operator that best represents the number of assets to be found, specify it with one of: >=, >, <, <= or ==

JSON Output Format:
- 'number_of_asset': [int - numerical value],
- 'asset_type': [string - type of asset]
- 'comparison_op': [string - type of comparison operator]

USER QUERY:
{}
""".format(QUERY)

llm = OpenAIChat(model_name="gpt-3.5-turbo", max_tokens=200, temperature=0)
response = llm(template)
response_json = json.loads(response)
print(response_json)

/Users/jai/dandi-search-response-ui/dandivenv/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{'number_of_asset': 2, 'asset_type': 'species', 'comparison_op': '>='}


In [17]:
import sys

# qdrant client
qdrant_client = Qdrant(
    location="https://906c3b3f-d3ff-4497-905f-2d7089487cf9.us-east4-0.gcp.cloud.qdrant.io", 
    port="6333",
    api_key=os.environ.get("QDRANT_API_KEY"),
)

# filter
num_asset: int = response_json.get("number_of_asset", None)
asset_type: str = response_json.get("asset_type", None)
comparison_op: str = response_json.get("comparison_op", None)

if not num_asset or not asset_type:
    print("Values not found.")
    sys.exit(0)
if not comparison_op:
    # default to exact value
    comparison_op = "=="

# determine asset type
if asset_type == "species":
    key = "number_of_species"
elif asset_type == "approach":
    key = "number_of_approaches"
elif asset_type == "variables_measured":
    key = "number_of_variables_measured"
elif asset_type == "measurement_techniques":
    key = "number_of_measurement_techniques"
else:
    key = None
if not key:
    print("Valid key not found")
    sys.exit(0)

# determine comparison opeartor
if comparison_op == "==":
    match = models.MatchValue(value=num_asset)
elif comparison_op == ">=":
    match = models.MatchAny(any=list(range(num_asset, 151)))
elif comparison_op == ">":
    match = models.MatchAny(any=list(range(num_asset + 1, 151)))
elif comparison_op == "<=":
    match = models.MatchAny(any=list(range(0, num_asset + 1)))
elif comparison_op == "<":
    match = models.MatchAny(any=list(range(0, num_asset)))
else:
    match = None

# qdrant filter
filter = models.Filter(must=[models.FieldCondition(key=key, match=match)])

# query similar results based on filter
docs = qdrant_client.scroll("dandi_collection_ada002", scroll_filter=filter, limit=100, with_vectors=False, with_payload=True)[0]
if not docs:
    print("No relevant dandisets found.")
else:
    for doc in docs:
        pl = doc.payload
        print(f"DANDI:{pl['dandiset_id']}/{pl['dandiset_version']}")
        for item in pl["species"]:
            print(f"- {item}")
        print("\n")

DANDI:000636/draft
- Human
- Homo sapiens - Human


DANDI:000212/draft
- Drosophila melanogaster - Fruit fly
- Drosophila suzukii


DANDI:000398/0.221208.1803
- Mus musculus - House mouse
- Rattus norvegicus - Norway rat


DANDI:000628/0.230922.1642
- Macaca mulatta - Rhesus monkey
- Macaca nemestrina - Pigtail macaque


DANDI:000341/draft
- Rattus norvegicus - Norway rat
- Homo sapiens - Human
- Mus musculus - House mouse


DANDI:000630/0.230915.2257
- Human
- Homo sapiens - Human


DANDI:000043/draft
- House mouse
- Human
- Rhesus monkey


